# Tema 3 - Exercitii `ipywidgets`

In [1]:
import ipywidgets as widgets
print(f'IPywidgets version: {widgets.__version__}')

import numpy as np
print(f'NumPy version: {np.__version__}')

import matplotlib.pyplot as plt
import matplotlib
print(f'Matplotlib version: {matplotlib.__version__}')

IPywidgets version: 7.6.3
NumPy version: 1.19.2
Matplotlib version: 3.3.4


In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
from typing import List

### Problema 1
Definiti o functie polinomiala de gradul 3, $f:\mathbb{R} \rightarrow \mathbb{R}$, cu coeficienti constanti prestabiliti. Aplicati algoritmul gradient descent pentru a vedea cum evolueaza cautarea minimului. Folositi minim doua controale ipywidgets: unul pentru pozitia initiala a lui $x$, altul pentru coeficientul $\alpha>0$ cu care se inmulteste gradientul. Gradientul va fi calculat analitic de voi sau folosind biblioteca [autograd](https://github.com/HIPS/autograd). 
Modificarea facuta prin metoda gradient descent este:
$$
x = x - \alpha \cdot f'(x)
$$
Se vor efectua minim 10 iteratii (optional: numarul de iteratii poate fi dat printr-un control ipywidgets), se vor marca pe grafic pozitiile succesive, in mod convenabil. 

In [3]:
f = lambda x: 3*x**3 + (-2)*x**2 + 2*x + 3
grad = lambda x: 9*x**2 + (-4)*x + 2

xf: np.ndarray = np.linspace(-10, 10, 100)
yf: np.ndarray = f(xf)

def sgd_plot(xf, yf, res=None, title = 'Functia $f(x)=3x^3 -2x^2 + 2x + 3$') -> None:
    """
    Function that displays the graph
    :param xf: an array that is the function domain(evenly spaced numbers over a specified interval)
    :param yf: an array that is the function codomain(the set of values from f(xf))
    :param res: specify if we want another function on the graph
    :param title: is the title of the function
    :returns: None
    """
    _, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(xf, yf)
    if res is not None:
        ax.plot(res, [f(x) for x in res], 'o')
    ax.set_title(title)
    ax.grid(axis='both')
    plt.show()

def sgd(x:int=10, alpha:float=0.001, epochs:int=10) -> List[float]:
    """
    Functions that searches for the minimum
    :param x: int number that represents the initial position
    :param alpha: float number that represents the scale factor from learning rate
    :param epochs: int that represents the number of iterations
    :param title: int number that represents the title of the function
    :returns: list of floats that represents the t
    """
    res: List[float] = [x]
    for _ in range(epochs):
        x -= alpha * grad(x)
        res += [x]
    return res

def gradient_descent(x=10, alpha=0.001, epochs=10) -> None:
    """
    Functions that calls the other 2 functions defined earlier. It helps us interact with the graph
    :param x: int number that represents the initial position
    :param alpha: float number that represents the scale factor from learning rate
    :param epochs: int that represents the number of iterations
    :returns: None
    """
    res:List[float] = sgd(x, alpha, epochs)
    sgd_plot(xf, yf, res, title=f'$f(x) = 3x^3 -2x^2 + 2x + 3$, start x={x}, alpha={alpha}')

    
interact(gradient_descent, x=(-10,10), alpha=(1e-3,5e-3,0.0001), epochs=(10,100));

interactive(children=(IntSlider(value=10, description='x', max=10, min=-10), FloatSlider(value=0.001, descript…

### Problema 2

Generati o lista de $n=100$ de perechi de valori $\{x_i, y_i\}_{i=0,n-1}$ in intervalul [-20, 10), afisati aceste valori pe un grafic, impreuna cu o dreapta definita de o functie liniara $y=a \cdot x+b$. Intr-un alt plot afisati, ca histograma, distanta dintre punctele de coordonate $(x_i, y_i)$ si punctele de intersectie ale verticalelor duse prin $x_i$ cu dreapta data, $\hat{y}_i$. Dreapta trebuie sa fie controlabila din widgets, prin cei doi coeficienti $a$ si $b$. Constatati modificarea histogramei in functie de pozitia dreptei si afisati mean squared error: $$MSE=\frac{1}{n} \cdot \sum_{i=0}^{n-1} (y_i - (a\cdot x_i + b))^2$$.
*Indicatii:*
1. Pentru generare de valori distribuite uniform in intervalul [0, 1) puteti folosi functia [numpy.random.uniform](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.uniform.html) si sa faceti inmultire si adunare in mod convenabil.
1. Puteti opta sa returnati cele $n$ puncte sub forma `vector_x`, `vector_y`.

In [4]:
def line(a:float=1, b:float=1) -> None:
    """
    Function that displays the graph, the distance between two coordinates and intersection points of verticals that goes through xi with the yi line
    :param a: first coefficient
    :param b: second coefficient
    :returns: None
    """
    x_rand: np.ndarray = np.random.uniform(-20, 10, 100)
    y_rand: np.ndarray = np.random.uniform(-20, 10, 100)
    range_x: np.ndarray = np.linspace(-20, 10, 100)
    values_y: np.ndarray = a * range_x + b
    
    predicted_y: np.ndarray = a * x_rand + b
        
    MSE = np.square(np.subtract(y_rand,predicted_y)).mean()
    
    vertical_distances = np.abs(y_rand - predicted_y)
    
    range_dist:int = int(max(vertical_distances) - min(vertical_distances)) + 1
    hist_bins: List[int] = [i for i in range(0, range_dist, range_dist//10+1)] + [range_dist]
        
    plt.figure(figsize=(10,5))
    plt.xlabel('x')
    plt.ylabel('y = ax + b')
    plt.plot(range_x, values_y, 'r', x_rand, y_rand, 'o')
    plt.grid(axis='both')
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.show()
    
    plt.hist(vertical_distances, bins=hist_bins)
    plt.xlabel('Vertical Distance')
    plt.ylabel('Number of appearances')
    plt.xlim([0, range_dist])
    plt.xticks(np.arange(0, range_dist+1, 3))
    plt.yticks(np.arange(0, 31, 5))
    plt.show()

    print(f'MSE = {MSE}')
    
interact(line, a=(-10, 10.0), b=(-10, 10.0));

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

### Problema 3
Incarcati fisierul `iris.data` din [setul de date iris](https://archive.ics.uci.edu/ml/datasets/iris). In functie de alegerile exprimate de un utilizator, afisati intr-un grafic 2D coloanele numerice alese (de exemplu, coloana 0 si coloana 2). Numele coloanelor se afla in fisierul iris.names.

*Indicatii/optiuni*:
* Incarcarea de date se poate face cu numpy, functia [loadtxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html). Specificati faptul ca se sare peste prima linie din fisier (header). Alternativ, puteti folosi [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
* Pentru cele doua alegeri puteti sa instantiati doua obiecte [Dropdown](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Dropdown) sau [Select](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Select).

In [6]:
data: np.ndarray = np.loadtxt("iris.data", delimiter=',', usecols=range(4))

sepal_dropdown = widgets.Dropdown(
    options = ['sepal length in cm', 'sepal width in cm', 'petal length in cm', 'petal width in cm'],
    description = 'First choice: ',
    value = 'sepal length in cm',
    disabled = False,
)
petal_dropdown = widgets.Dropdown(
    options = ['sepal length in cm', 'sepal width in cm', 'petal length in cm', 'petal width in cm'],
    description = 'Second choice: ',
    value = 'petal length in cm',
    disabled = False,
)

def plot_it(sepal_dropdown, petal_dropdown):
    """
    Function that displays the graph with a cloud of dots representing length and width of an iris flower
    :param sepal_dropdown: a dropdown representing length/width of a sepal
    :param petal_dropdown: a dropdown representing length/width of a petal
    :returns: None
    """
    plt.title("Iris Data Set")
    plt.xlabel('Iris-setosa                Iris-versicolor                 Iris-virginica')
    plt.ylabel('Centimeters')
    plt.xlim(0, data.shape[0])
    plt.xticks(np.arange(0, data.shape[0]+1, 10))
    
    if sepal_dropdown == "sepal length in cm":
        col: np.ndarray = data[:, 0]
        plt.plot(col, 'r*', label='Sepal length in cm')
    elif sepal_dropdown == 'sepal width in cm':
        col: np.ndarray = data[:, 1]
        plt.plot(col, 'gv', label='Sepal width in cm')
    elif sepal_dropdown == "petal length in cm":
        col: np.ndarray = data[:, 2]
        plt.plot(col, 'b^', label='Petal length in cm')
    else:
        col: np.ndarray = data[:, 3]
        plt.plot(col, 'c+', label='Petal width in cm')
        
    if petal_dropdown == "sepal length in cm":
        col: np.ndarray = data[:, 0]
        plt.plot(col, 'r*', label='Sepal length in cm')
    elif petal_dropdown == 'sepal width in cm':
        col: np.ndarray = data[:, 1]
        plt.plot(col, 'gv', label='Sepal width in cm')
    elif petal_dropdown == "petal length in cm":
        col: np.ndarray = data[:, 2]
        plt.plot(col, 'b^', label='Petal length in cm')
    else:
        col: np.ndarray = data[:, 3]
        plt.plot(col, 'c+', label='Petal width in cm')
    plt.legend()
    
interact(plot_it, sepal_dropdown=sepal_dropdown, petal_dropdown=petal_dropdown);

interactive(children=(Dropdown(description='First choice: ', options=('sepal length in cm', 'sepal width in cm…

### Problema 4
Generati $n$ perechi de puncte aleatoare, folosind o functie $f:\mathbb{R} \rightarrow \mathbb{R}$ de alease e voi (de exemplu: functie polinomiala + zgomot aleator adaugat). Alegeti 5 metode de interpolare din [scipy.interpolate](https://docs.scipy.org/doc/scipy/reference/interpolate.html) si reprezentati grafic valorile interpolate. Folositi controale ipywidgets cel putin pentru alegerea lui $n$ si a metodei de interpolare aleasa.

In [9]:
from scipy import interpolate
from scipy.interpolate import barycentric_interpolate, pchip_interpolate, krogh_interpolate, CubicSpline

interpolation_dropdown = widgets.Dropdown(
    options = ["1-D Interpolation", "Barycentric Interpolation", "Pchip Interpolation", "Krogh Interpolation", "Cubic Spline Interpolation"],
    description = "Interpolation Method: ",
    value = '1-D Interpolation',
    disabled = False,
)
def funct(interpolation_dropdown, n=50) -> None:
    """
    Function that displays the graph with the result of the function
    :param n: int that represents how many pairs we generate
    :param interpolation_dropdown: a dropdown representing the type of interpolation
    :returns: None
    """
    range_x: np.ndarray = np.linspace(-10,10,n)
    values_y: np.ndarray = 2*range_x**3 + 2*range_x**2 - 3*range_x + 7
    
    plt.figure(figsize=(10,5))
    plt.xlabel('x')
    plt.ylabel('y = $-3x^2 + 6x -7$')
    plt.grid(axis='both')
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    if interpolation_dropdown == '1-D Interpolation':
        f = interpolate.interp1d(range_x, values_y)
        xnew: np.ndarray = np.arange(-10,10,0.5)
        ynew: np.ndarray = f(xnew)
        plt.plot(range_x, values_y, 'r-', label='observation')
        plt.plot(xnew, ynew, 'bo', label='1-D Interpolation')
        plt.legend()
    elif interpolation_dropdown == "Barycentric Interpolation":
        x: np.ndarray = np.linspace(min(range_x), max(range_x), num=10)
        y: np.ndarray = barycentric_interpolate(range_x, values_y, x)
        plt.plot(range_x, values_y, 'r-', label="observation")
        plt.plot(x, y, 'bo', label='Barycentric Interpolation')
        plt.legend()
    elif interpolation_dropdown == "Pchip Interpolation":
        x: np.ndarray = np.linspace(min(range_x), max(range_x), num=10)
        y: np.ndarray = pchip_interpolate(range_x, values_y, x)
        plt.plot(range_x, values_y, 'r-', label='observation')
        plt.plot(x, y, 'bo', label='pchip Interpolation')
        plt.legend()
    elif interpolation_dropdown == "Krogh Interpolation":
        x: np.ndarray = np.linspace(min(range_x), max(range_x), num=10)
        y: np.ndarray = krogh_interpolate(range_x, values_y, x)
        plt.plot(range_x, values_y, 'r-', label='observation')
        plt.plot(x, y, 'bo', label='Krogh Interpolation')
        plt.legend()
    else:
        cs = CubicSpline(range_x, values_y)
        xs: np.ndarray = np.arange(-10, 10, 0.5)
        plt.plot(range_x, values_y, 'r-', label='observation')
        plt.plot(xs, cs(xs), 'bo', label='Cubic Spline Interpolation')
#         plt.plot(xs, cs(xs, 1), 'b-', label = "First Derivative")
#         plt.plot(xs, cs(xs, 2), 'y-', label = "Second Derivative")
#         plt.plot(xs, cs(xs, 3), 'c-', label = 'Third Derivative')
        plt.legend()
        
interact(funct, n=(10, 100), interpolation_dropdown=interpolation_dropdown);


interactive(children=(Dropdown(description='Interpolation Method: ', options=('1-D Interpolation', 'Barycentri…